In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import PCA

In [3]:
sentiments = pd.read_csv('./data/judge-1377884607_tweet_product_company.csv',encoding='ISO-8859-1')
sentiments.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,NaN,No emotion toward brand or product


In [4]:
sentiments['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [111]:
sentiments['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [5]:
sentiments.dropna(subset=['tweet_text'],inplace=True)

In [110]:
def tokenize(text):
    sw = stopwords.words('english')
    text = [str(phrase).lower() for phrase in text]
    tokenizer_ = RegexpTokenizer(r"([@#]?[a-zA-Z]+(?:’[a-z]+)?)")
    tokens = [' '.join(tokenizer_.tokenize(str(doc))) for doc in text]
    return tokens

In [38]:
TfidfTransformer?

Init signature:
TfidfTransformer(
    *,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)
Docstring:     
Transform a count matrix to a normalized tf or tf-idf representation

Tf means term-frequency while tf-idf means term-frequency times inverse
document-frequency. This is a common term weighting scheme in information
retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a
token in a given document is to scale down the impact of tokens that occur
very frequently in a given corpus and that are hence empirically less
informative than features that occur in a small fraction of the training
corpus.

The formula that is used to compute the tf-idf for a term t of a document d
in a document set is tf-idf(t, d) = tf(t, d) * idf(t), and the idf is
computed as idf(t) = log [ n / df(t) ] + 1 (if ``smooth_idf=False``), where
n is the total number of documents in the document set

In [61]:
sw = stopwords.words('english')
# cvr = CountVectorizer(
#     encoding='ISO-8859-1',
#     stop_words=sw,
#     ngram_range=[2,4],
#     min_df=4,max_df=50
#     )

In [62]:
emotion  = 'is_there_an_emotion_directed_at_a_brand_or_product'
directed = 'emotion_in_tweet_is_directed_at'

In [101]:
y_n = sentiments
[
                (sentiments[emotion] == 'Positive emotion') |
                (sentiments[emotion] == 'Negative emotion')
                ]

[0        True
 1        True
 2        True
 3        True
 4        True
         ...  
 9088     True
 9089    False
 9090    False
 9091    False
 9092    False
 Name: is_there_an_emotion_directed_at_a_brand_or_product, Length: 9092, dtype: bool]

In [64]:
y_n[emotion].value_counts()

Positive emotion    2978
Negative emotion     570
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [112]:
tokens

['. @wesley83 i have a 3g iphone . after 3 hrs tweeting at #rise_austin , it was dead ! i need to upgrade . plugin stations at #sxsw .',
 "@jessedee know about @fludapp ? awesome ipad / iphone app that you'll likely appreciate for its design . also , they're giving free ts at #sxsw",
 '@swonderlin can not wait for #ipad 2 also . they should sale them down at #sxsw .',
 "@sxsw i hope this year's festival isn't as crashy as this year's iphone app . #sxsw",
 "@sxtxstate great stuff on fri #sxsw : marissa mayer ( google ) , tim o'reilly ( tech books / conferences ) & matt mullenweg ( wordpress )",
 '@teachntech00 new ipad apps for #speechtherapy and communication are showcased at the #sxsw conference http://ht.ly/49n4m #iear #edchat #asd',
 '#sxsw is just starting , #ctia is around the corner and #googleio is only a hop skip and a jump from there , good time to be an #android fan',
 'beautifully smart and simple idea rt @madebymany @thenextweb wrote about our #hollergram ipad app for #sxsw

In [107]:
# X = cvr.fit_transform(sentiments['tweet_text'])
# tokens = tokenize(sentiments['tweet_text'])
tokens = tokenize(y_n['tweet_text'])
# X = cvr.fit_transform(tokens)
# X
lemmatizer = WordNetLemmatizer()
lem = []
for doc in tokens:
    lem.append(' '.join([lemmatizer.lemmatize(token) for token in doc.split(' ')]))
tfidf = TfidfVectorizer(ngram_range=[1,4],max_df=0.5,min_df=10)
# print(tokens)
X = tfidf.fit_transform(lem)

In [108]:
y_n['tweet_text']

0       .@wesley83 I have a 3G iPhone. After 3 hrs twe...
1       @jessedee Know about @fludapp ? Awesome iPad/i...
2       @swonderlin Can not wait for #iPad 2 also. The...
3       @sxsw I hope this year's festival isn't as cra...
4       @sxtxstate great stuff on Fri #SXSW: Marissa M...
                              ...                        
9088                        Ipad everywhere. #SXSW {link}
9089    Wave, buzz... RT @mention We interrupt your re...
9090    Google's Zeiger, a physician never reported po...
9091    Some Verizon iPhone customers complained their...
9092    Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...
Name: tweet_text, Length: 9092, dtype: object

In [109]:
vect = pd.DataFrame(X.toarray(),columns=tfidf.get_feature_names())
vect.head()

,00,00 11,00 11 59,00 11 59 59,00 59,00 59 59p,00 59 59p 03,000,03,03 11,...,ûªre,ûªs,ûªt,ûï,ûï mention,ûï mention apple,ûï mention google,ûï mention google to,ûò,ûó
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
tfidf.get_feature_names()

['about',
 'about the',
 'about to',
 'aclu',
 'action',
 'action link',
 'actually',
 'ad',
 'after',
 'again',
 'agree',
 'all',
 'all sxsw',
 'all the',
 'almost',
 'already',
 'also',
 'always',
 'am',
 'amazing',
 'america',
 'among',
 'amp',
 'amp ipad',
 'amp ipad launch',
 'amp mention',
 'amp physical',
 'amp physical world',
 'amp physical world through',
 'an',
 'an apple',
 'an ipad',
 'an ipad at',
 'an ipad at sxsw',
 'an ipad link',
 'an ipad sxsw',
 'an iphone',
 'analytics',
 'and',
 'and android',
 'and congress',
 'and get',
 'and google',
 'and ipad',
 'and it',
 'and mention',
 'and the',
 'and the ipad',
 'and the ipad launch',
 'andoid',
 'andoid app',
 'andoid app at',
 'andoid app at the',
 'android',
 'android app',
 'android choice',
 'android choice award',
 'android choice award thanks',
 'android is',
 'android link',
 'android market',
 'android party',
 'android phone',
 'another',
 'any',
 'anyone',
 'anything',
 'app',
 'app at',
 'app at sxsw',
 'app 

In [29]:
PCA?

Init signature:
PCA(
    n_components=None,
    *,
    copy=True,
    whiten=False,
    svd_solver='auto',
    tol=0.0,
    iterated_power='auto',
    random_state=None,
)
Docstring:     
Principal component analysis (PCA).

Linear dimensionality reduction using Singular Value Decomposition of the
data to project it to a lower dimensional space. The input data is centered
but not scaled for each feature before applying the SVD.

It uses the LAPACK implementation of the full SVD or a randomized truncated
SVD by the method of Halko et al. 2009, depending on the shape of the input
data and the number of components to extract.

It can also use the scipy.sparse.linalg ARPACK implementation of the
truncated SVD.

Notice that this class does not support sparse input. See
:class:`TruncatedSVD` for an alternative with sparse data.

Read more in the :ref:`User Guide <PCA>`.

Parameters
----------
n_components : int, float, None or str
    Number of components to keep.
    if n_components is not 

In [71]:
pca = PCA(n_components=0.85)

In [104]:
pca.fit_transform(vect,y_n[emotion])

array([[-0.03878556, -0.03052832, -0.02775639, ..., -0.02370225,
         0.01366001,  0.02739418],
       [-0.05399023, -0.06821232,  0.00311386, ...,  0.00921313,
        -0.0204802 ,  0.024614  ],
       [-0.04467274, -0.01021552, -0.01989074, ...,  0.00165331,
        -0.00711658,  0.01523277],
       ...,
       [-0.02039962, -0.13032255,  0.06472398, ...,  0.02168409,
        -0.03677948, -0.00680019],
       [-0.02876431, -0.05596251, -0.00916069, ..., -0.00778626,
         0.00533722,  0.00487221],
       [ 0.02908096, -0.01373481, -0.01404286, ..., -0.00307387,
         0.00348966,  0.00710547]])

In [90]:
[np.argmax(pca.components_[0])]

[515]

In [105]:
pca.explained_variance_ratio_.cumsum()

array([0.02296384, 0.03387096, 0.04157291, ..., 0.84964519, 0.84983738,
       0.85002932])

In [91]:
len(pca.components_)

559

In [94]:
display(sentiments.head(10), y_n.head())

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion
10,Excited to meet the @samsungmobileus at #sxsw ...,Android,Positive emotion


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [97]:
sentiments.sample(5)['tweet_text']

275     ÛÏ@mention Apple is a great example of a reta...
5312    RT @mention #SXSW Pro Tip: Remove access to co...
1829    Awww yeaaa new music from @mention just in tim...
8472    Where is this ipad pop up store? #sxsw #caniex...
957     ÛÏ@mention Official #SXSW App Û÷SXSW GOÛª {...
Name: tweet_text, dtype: object

In [99]:
sentiments.iloc[5014]['tweet_text']

'save me some cash! TechCrunch Giveaway: An iPad 2åÊ#TechCrunch {link} via @mention #winning! #ipad2 #sxsw #apple'

In [100]:
for col in ['emotion_in_tweet_is_directed_at','is_there_an_emotion_directed_at_a_brand_or_product']:
    display(sentiments[col].value_counts())
sentiments.info()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
dtypes: object(3)
memory usage: 284.1+ KB


In [83]:
# sentiments.isna()
vals_ = sentiments[sentiments['emotion_in_tweet_is_directed_at'].isna()].value_counts('is_there_an_emotion_directed_at_a_brand_or_product')
vals_

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5297
Positive emotion                       306
I can't tell                           147
Negative emotion                        51
dtype: int64

In [84]:
sentiments[sentiments['tweet_text'].isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product


In [85]:
vals_.values.sum()

5801

In [86]:
vect.head()

,about,about the,about to,aclu,action,action link,actually,ad,after,again,...,you speak quot,you speak quot mark,you tweet,you tweet quot,you tweet quot is,your,your ipad,your iphone,yrs,zazzlesxsw
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.348367,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.196329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
